In [41]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
from torch.optim import SGD
from torch import nn

## Access src directory from ./notebooks/ folder
from pathlib import Path
sys.path.append(str(Path('.').absolute().parent))

from src.Snip import SNIP
from src.tools import n_layer_nn
from src.models import ConvNet

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Manual test with F-MNIST

In [42]:
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms

In [43]:
batch_size = 64
path = '../data/'
dataset = FashionMNIST(path, train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor()
                        ]))
data_loader = torch.utils.data.DataLoader(dataset,
                        batch_size=batch_size, shuffle=True, num_workers=2)

In [59]:
model = ConvNet()
snip = SNIP(model)
C = snip.prune(nn.CrossEntropyLoss(),data_loader, 8)
#for elem in C:
#    print(elem.shape)
#sgd_val_loss, sgd_accuracy = n_layer_nn(SGD, learning_rate=0.1)

#C = snip.prune()
# to multiply weights by C : https://discuss.pytorch.org/t/how-can-i-modify-certain-layers-weight-and-bias/11638
C.sum()

/home/samutamm/Documents/Study/Sorbonne/AS/project/src/Snip.py:10: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


8.0

9.0

In [32]:
dimensions = [2,2]
model.state_dict()

OrderedDict([('features.0.weight',
              tensor([[[[-0.1072, -0.0214, -0.0762,  0.1586, -0.0889],
                        [-0.1492, -0.0926, -0.1198,  0.1367,  0.0172],
                        [ 0.1799, -0.0617, -0.0060, -0.1381, -0.0992],
                        [-0.0441, -0.1085,  0.0914, -0.0808,  0.0750],
                        [-0.0715, -0.0934,  0.0346,  0.1705, -0.0816]]],
              
              
                      [[[-0.0079, -0.0777, -0.0623, -0.0718,  0.1411],
                        [ 0.0228, -0.0833, -0.1392, -0.0206, -0.0349],
                        [ 0.0462, -0.1320, -0.0500, -0.1407,  0.1315],
                        [-0.1344,  0.0016, -0.1638, -0.0838, -0.1248],
                        [ 0.1340, -0.1232, -0.1200,  0.1535,  0.1313]]],
              
              
                      [[[ 0.1495, -0.0748, -0.0306, -0.0268, -0.0782],
                        [ 0.0977,  0.0083, -0.0162,  0.0951,  0.0520],
                        [-0.1662,  0.1048,  0.127

In [23]:
np.unravel_index(0, (6,1,5,5))

(0, 0, 0, 0)

In [35]:
params, params_id_mapping = snip.get_all_parameters()

In [39]:
params_id_mapping

{0: (0, (0, 0, 0, 0)),
 1: (0, (0, 0, 0, 1)),
 2: (0, (0, 0, 0, 2)),
 3: (0, (0, 0, 0, 3)),
 4: (0, (0, 0, 0, 4)),
 5: (0, (0, 0, 1, 0)),
 6: (0, (0, 0, 1, 1)),
 7: (0, (0, 0, 1, 2)),
 8: (0, (0, 0, 1, 3)),
 9: (0, (0, 0, 1, 4)),
 10: (0, (0, 0, 2, 0)),
 11: (0, (0, 0, 2, 1)),
 12: (0, (0, 0, 2, 2)),
 13: (0, (0, 0, 2, 3)),
 14: (0, (0, 0, 2, 4)),
 15: (0, (0, 0, 3, 0)),
 16: (0, (0, 0, 3, 1)),
 17: (0, (0, 0, 3, 2)),
 18: (0, (0, 0, 3, 3)),
 19: (0, (0, 0, 3, 4)),
 20: (0, (0, 0, 4, 0)),
 21: (0, (0, 0, 4, 1)),
 22: (0, (0, 0, 4, 2)),
 23: (0, (0, 0, 4, 3)),
 24: (0, (0, 0, 4, 4)),
 25: (0, (1, 0, 0, 0)),
 26: (0, (1, 0, 0, 1)),
 27: (0, (1, 0, 0, 2)),
 28: (0, (1, 0, 0, 3)),
 29: (0, (1, 0, 0, 4)),
 30: (0, (1, 0, 1, 0)),
 31: (0, (1, 0, 1, 1)),
 32: (0, (1, 0, 1, 2)),
 33: (0, (1, 0, 1, 3)),
 34: (0, (1, 0, 1, 4)),
 35: (0, (1, 0, 2, 0)),
 36: (0, (1, 0, 2, 1)),
 37: (0, (1, 0, 2, 2)),
 38: (0, (1, 0, 2, 3)),
 39: (0, (1, 0, 2, 4)),
 40: (0, (1, 0, 3, 0)),
 41: (0, (1, 0, 3, 1)),
 4